# Data cleaning

In [57]:
import pandas as pd

sin_bandera = pd.read_csv('data/sin_bandera.csv')
sin_bandera['language'] = 'Spanish'
loveless = pd.read_csv('data/loveless.csv')
loveless['language'] = 'English'

songs = pd.concat([sin_bandera, loveless], ignore_index=True)
print(songs.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist        20 non-null     object
 1   title         20 non-null     object
 2   release_date  20 non-null     object
 3   pageviews     20 non-null     int64 
 4   album         20 non-null     object
 5   lyrics        20 non-null     object
 6   language      20 non-null     object
dtypes: int64(1), object(6)
memory usage: 1.2+ KB
None


## Processing the lyrics
The process of song lyrics focuses on removing punctuaction and non-informative words. The first filter consisted in using regular expressions on the text formated as lower to remove expressions similar to 'ah', 'oh' or 'mm' and then to keep only words in Spanish and English.

The second step used `nltk` (Natural Language Toolkit), a library used for working with humane language data, capable of removing words that don't carry significant meaning through its predefined list of `stopwords`. Examples are 'the', 'is', 'es', 'un' and similar.

The defined function, `clean_lyrics`, takes raw `text` and `language` as arguments and performs the cleaning, also removing words with less than 3 letters as a final filter before returning the final result.

In [ ]:
import re
from nltk.corpus import stopwords

# Process the lyrics with regular expressions and stopwords from nltk
# Run nltk.download('stopwords') once before calling the function and 
# when using nltk for the first time, not needed again.
def clean_lyrics(text, language='Spanish'):
    # Remove expressions similar to 'oh'
    text = re.sub(r'\b([oaeui][h]?|la|na|mm|ja)\1+\b', '', text.lower())
    # Keep only letters and spaces (accents included)
    text = re.sub(r'[^\w\sáéíóúüñ]', ' ', text)

    if language == 'Spanish':
        stops = set(stopwords.words('spanish') + ['yeah', 'yeh'])
    else:
        stops = set(stopwords.words('english') + ['yeah','mmh', 'bam', 'ooh'])
    
    words = [w for w in text.split() if w not in stops and len(w) > 2]
    
    return ' '.join(words)

songs['clean_lyrics'] = songs.apply(lambda row: clean_lyrics(row['lyrics'], language=row['language']), axis=1)

print(songs['clean_lyrics'][0])

contributorsentra vida lyrics buenas noches gusto chica después cinco minutos alguien especial hablarme tocarme dentro encendió ojos hacía tarde olvidaba reloj días lado enseñaron verdad tiempo determinado comenzar amar siento tan profundo explicación razón lógica corazón entra vida abro puerta brazos noches desiertas entra vida ruego comencé extrañar empecé necesitarte luego buenas noches gusto existe nadie después tiempo juntos puedo volver atrás hablaste tocaste volviste ilusión quiero dueña corazón entra vida abro puerta brazos noches desiertas entra vida ruego comencé extrañar empecé necesitarte luego entra horas sálvame ahora abre brazos fuerte déjame entrar entra vida abro puerta brazos noches desiertas noches desiertas entra vida ruego ruego comencé extrañar empecé necesitarte luego comencé extrañar empecé necesitarte luego


## Removing some undesired info

Information like the title and a brief description of the artist and the song is present in lyrics and it's not completely removed by the application of the previous function. For example, the last output has the following text in the beggining: "contributorsentra vida lyrics". After reviewing the outputs for other songs, I identified a repetitive pattern where the desired lyric always started after the word "lyrics". The function below removes the text that is before that word. 

In [ ]:
# A function to isolate 'lyrics' and delete what is before
def after_lyrics(text):
    '''
    Some undesired information is still present in the values returned by clean_lyrics.
    A repetitive pattern exist in them, where the desired lyric always starts after the word "lyrics".
    "lyrics" can be a single word or be present in an expression like "otherwordlyrics", so regex is useful.
    This function removes what is before that word, including that word.
    '''
    split = text.split()
    required_index = 0
        
    for expression in split:
        # Search if pattern exists anywhere
        if re.search(r'lyrics', expression):
            required_index = split.index(expression)
        
    # Update the expression
    split[required_index] = 'lyrics'
    # Get the index to ignore what is before
    get_index = split.index('lyrics')
    return ' '.join(split[get_index + 1:])

In [ ]:
# Improved version
t = 'contributorsentra vida lyrics buenas noches gusto'
t2 = 'contributorsentra vida thelyrics buenas noches gusto'

def after_lyrics(text):
    split = text.split()
        
    for expression in split:
        # Search if pattern exists anywhere
        if re.search(r'lyrics', expression):
            # Get the index to ignore what is before
            get_index = split.index(expression) 
            return ' '.join(split[get_index + 1:])
    
    return ' '.join(split)

print(after_lyrics(t))
print(after_lyrics(t2))

buenas noches gusto
buenas noches gusto


In [60]:
# Remove the titles and words before 'lyrics'
songs['clean_lyrics'] = songs['clean_lyrics'].apply(after_lyrics)
print(songs['clean_lyrics'].iloc[-1])

called drunk benz driving home influence scared death wasting breath cause listen fucking friends relate relate cause never treat shitty made hate city talk shit internet never told anyone anything bad cause shit embarrassing everything make fucking sad waste time try make feel bad could talk every time showed time empty line cause never never paid mind mother friends shut cause kid ruined everything good always said misunderstood made moments fucking leave alone relate relate cause never treat shitty make hate city


## Replacing a word with its synonym

The Spanish word "tal vez" can be interpreted as two separated words, causing problems with further analysis like word frequencies. The approach here consisted in replacing the term with a synonym, "quizás", only for the Spanish lyrics.

In [ ]:
# Logic
# Modification: quizás instead of tal vez
string = 'tal vez, si tal vez quisieras cambiar, quizás'
# Keep only words
string = re.sub(r'[^\w\sáéíóúüñ]', '', string)
split = string.split()
print(split)

# Modify the split list
for word in split:
    if word == 'tal':
        next_word_index = split.index(word) + 1 # Next word index
        if split[next_word_index] == 'vez':
            split[next_word_index - 1] = 'quizás' # Replace 'tal' for 'quizás'
            del split[next_word_index] # Delete 'vez', that remains at the same position

print(split)

['tal', 'vez', 'si', 'tal', 'vez', 'quisieras', 'cambiar', 'quizás']
['quizás', 'si', 'quizás', 'quisieras', 'cambiar', 'quizás']


In [ ]:
# Functional application
def del_tal_vez(lyrics):
    '''
    The Spanish word "tal vez" can be interpreted as two separated words,
    causing problems with further analysis like word frequencies.
    The approach here consisted in replacing the term with a synonim: "quizás"
    '''
    split = lyrics.split()
    # Modify the split list
    for word in split:
        if word == 'tal':
            next_word_index = split.index(word) + 1 # Next word index
            try: # Handle 'tal' as last word of the lyric
                if split[next_word_index] == 'vez':
                    split[next_word_index - 1] = 'quizás' # Replace 'tal' for 'quizás'
                    del split[next_word_index] # Delete 'vez', that remains at the same position
            except IndexError:
                pass
        
    return ' '.join(split)

string = 'tal si tal vez quisieras cambiar quizás'
str2 = 'tal vez si tal vez quisieras cambiar quizás tal'
print(del_tal_vez(string))
print(del_tal_vez(str2))

tal si tal vez quisieras cambiar quizás
quizás si quizás quisieras cambiar quizás tal


In [63]:
# Apply to the clean_lyrics column, filtering for Spanish only
songs.loc[songs['language'] == 'Spanish', 'clean_lyrics'] = \
songs.loc[songs['language'] == 'Spanish', 'clean_lyrics'].apply(del_tal_vez)

print(songs['clean_lyrics'].iloc[1])

quédate momento así mires hacia podré aguantar clavas mirada hiela cuerpo pasado puedo hablar quizás pienses loco verdad aceptar explico siento dentro vas entender veas llorar nunca sentí tan solo ayer pronto entendí mientras callaba vida dijo gritos nunca nunca perdí explicaba amor cosa pronto forma natural lleno fuego fuerzas marchita tener principio llega final ahora quizás puedas entender tocas quema piel ahora quizás puedas entender vuelvas quieres ver lloro lloro entendí lloro nunca sentí tan solo ayer pronto entendí mientras callaba vida dijo gritos nunca nunca perdí explicaba amor cosa pronto forma natural lleno fuego fuerzas marchita tener principio llega final ahora quizás puedas entender tocas quema piel ahora quizás puedas entender vuelvas quieres ver lloro lloro entendí lloro lloro lloro entendí lloro


## Performing a specific fixing

The song "En Esta No" by Sin Bandera has a unique issue related to undesired information contained in lyrics. The word "lyrics" doesn't appear here and the function `after_lyrics` was not able to clean it, so a specific fixing was applied.

In [64]:
# Song 'En Ésta No' at index 5 has information not related to the lyrics, so it will be removed
print(songs.iloc[5, 7])

lyrics = songs.iloc[5, 7].split()
index_to_slice = lyrics.index('more')
lyrics = lyrics[index_to_slice + 1:]
updated_lyrics = ' '.join(lyrics)

songs.iloc[5, 7] = updated_lyrics
print('Modified:')
print(songs.iloc[5, 7])

song latin duo bandera formed mexican singer leonel garcía and argentinean singer noel schajris noel and leonel prepare launch february with songs ésta the first read more toca ser ama toca hacer juntos cama dar cuerda reloj coinciden universos podemos escribir verso describa amor toca caminar mundo viajar profundo cielo abrió historia nunca comenzó quizás vida pueda darte siento ahora quizás vida toque cuerpo contemplar aurora quizás vida cante piel misma voz quizás vida beba boca todas ansias quizás vida amor distante acorte distancias quizás vida luz quizás vida primero vida toca decirnos quiero cuidar dinero quedado cajón aunque duela aceptarlo quede ganas dar quema corazón historia nunca comenzó quizás vida pueda darte siento ahora quizás vida toque cuerpo contemplar aurora quizás vida cante piel misma voz quizás vida beba boca todas ansias quizás vida amor distante acorte distancias quizás vida luz quizás vida primero vida quizás vida luz quizás vida primero vida vida
Modified:
t

## Modification of the cleaning functions

After doing some research and looking for analysis techniques to apply on the clean data, I encountered an approach that consisted in measuring the ryme of the songs. It's simple and creates an score by comparing the last two letters of adjacent words, evaluating whether they're equal. In doing so, I realised that the words I should focus on were the ones at the end of a verse. The original data I retrieved kept a verse-like structure by using newlines (`"\n"`), so I modified the previous `clean_lyrics` function to keep them.

A new function to remove unnecessary information was also created, derived from `after_lyrics`, because I noted that it appeared in the first element of the split done with newlines instead of spaces.

In [65]:
# Unprocessed lyrics
raw = songs['lyrics'].iloc[2]
print(raw)

4 ContributorsMientes Tan Bien Lyrics

Que te quedarás conmigo una vida entera
Que contigo: adiós inviernos, solo primavera
Que las olas son de magia y no de agua salada
Yo te creo todo y tú no me das nada, tú no me das nada

Que si sigo tu camino, llegaré hasta el cielo
Tú me mientes en la cara y yo me vuelvo ciego
Yo me trago tus palabras, tú juegas un juego
Y me brilla el mundo cuando dices "luego"
Cuando dices "luego"

Cuando dices: "Siento, siento que eres todo"
Cuando dices: "Vida, yo estaré contigo"
Tomas de mi mano, y por dentro lloro
Aunque sea mentira, me haces sentir vivo
Aunque es falso el aire, siento que respiro

Mientes tan bien
Que me sabe a verdad todo lo que me das
Y ya te estoy amando
Mientes tan bien
Que he llegado a imaginar que en mi amor llenas tu piel
Y aunque todo es de papel, mhm, mientes tan bien

Cuando dices: "Siento, siento que eres todo"
Cuando dices: "Vida, yo estaré contigo"
Tomas de mi mano y por dentro lloro
Aunque sea mentira, me haces sentir vivo
Au

In [ ]:
# A different cleaning, keeping all the words and newlines
def clean_modified(text):
    # Remove expressions similar to 'oh'
    text = re.sub(r'\b(?:a+h+|o+h+||u+h+|e+h+|mm+|ja+|la+|na+)\b', '', text.lower())
    # Keep only letters, spaces and new lines
    text = re.sub(r"[^\wáéíóúüñ'\n]", ' ', text)
    # Collapse multiple spaces into one
    text = re.sub(r' +', ' ', text)
    
    return '\n'.join([line.strip() for line in text.split('\n') if line.strip()])

# A function to delete unnecessary information from lyrics, similar to after_lyrics
def after_lyrics_modified(text):
    try:
        split = text.split('\n') # Split line by line, not word by word
        del split[0] # Delete the first element: where not needed data is
        return '\n'.join(split)
    except:
        return text

# Ckeck the result: working as expected
print(after_lyrics_modified(clean_modified(raw)))

que te quedarás conmigo una vida entera
que contigo adiós inviernos solo primavera
que las olas son de magia y no de agua salada
yo te creo todo y tú no me das nada tú no me das nada
que si sigo tu camino llegaré hasta el cielo
tú me mientes en cara y yo me vuelvo ciego
yo me trago tus palabras tú juegas un juego
y me brilla el mundo cuando dices luego
cuando dices luego
cuando dices siento siento que eres todo
cuando dices vida yo estaré contigo
tomas de mi mano y por dentro lloro
aunque sea mentira me haces sentir vivo
aunque es falso el aire siento que respiro
mientes tan bien
que me sabe a verdad todo lo que me das
y ya te estoy amando
mientes tan bien
que he llegado a imaginar que en mi amor llenas tu piel
y aunque todo es de papel mhm mientes tan bien
cuando dices siento siento que eres todo
cuando dices vida yo estaré contigo
tomas de mi mano y por dentro lloro
aunque sea mentira me haces sentir vivo
aunque es falso el aire siento que respiro
mientes tan bien
que me sabe a verda

In [ ]:
# clean_modified application
check = songs['lyrics'].iloc[6]
print(clean_modified(check))

4 contributorssuelta mi mano lyrics
no no es necesario que lo entienda
porque nunca le ha servido razón al corazón el corazón no piensa
no mi vida para qué te esfuerzas no me tienes que explicar
siempre amaré tu libertad por mucho que eso duela
y sí entiendo que quieres hablar
que a veces necesitas saber de mí
pero no sé si quiera saber de ti
vivir así seguir así pensando en ti
suelta mi mano ya por favor
entiende que me tengo que ir
si ya no sientes más este amor
no tengo nada más que decir
no digas nada ya por favor
te entiendo pero entiéndeme a mí
cada palabra aumenta el dolor
y una lágrima quiere salir
y por favor no me detengas
siempre encuentro manera de seguir y de vivir aunque ahora no tenga
y no mi vida no vale pena
para qué quieres llamar si el que era yo ya no va a estar
esta es última cena
y sí entiendo que quieres hablar
que a veces necesitas saber de mí
pero no sé si quiera saber de ti
vivir así seguir así pensando en ti
suelta mi mano ya por favor
entiende que me tengo q

The final result was suitable to perform the analysis of rhyme. I also decided to keep all the words from the original lyrics by removing the code that executed the stopwords deleting in `clean_modified` because of the rhyme too. However, the first cleaning performed by `clean_lyrics` and the original lyrics were kept too in the final dataset saved as `.csv`.

In [68]:
# Applying it to the dataset
songs['lyrics'] = songs['lyrics'].apply(clean_modified)
# Remove the titles and words before 'lyrics' too
songs['lyrics'] = songs['lyrics'].apply(after_lyrics_modified)

# Handle the special observation
lyrics = songs.iloc[5, 5].split('\n')
lyrics = lyrics[1:] # Because the last cleaning already removed one element
updated_lyrics = '\n'.join(lyrics)

songs.iloc[5, 5] = updated_lyrics
print(songs.iloc[5, 5])

en esta no
no me toca ser el que te ama
ni nos toca hacer juntos cama
ni dar cuerda a este reloj
en esta no
no coinciden nuestros universos
ni podemos escribir un verso
que describa nuestro amor
en esta no
no nos toca caminar el mundo
ni viajar hasta lo más profundo
de este cielo que se abrió
en esta no
nuestra historia nunca comenzó
tal vez en otra vida
pueda darte todo lo que siento ahora
tal vez en otra vida
me toque en tu cuerpo contemplar aurora
tal vez en otra vida seamos tú y yo
y cante nuestra piel con una misma voz
tal vez en otra vida
beba de tu boca todas esas ansias
tal vez en otra vida
este amor distante acorte las distancias
tal vez en otra vida se nos dé luz
tal vez en otra vida seas primero tú
en esta vida no
en esta no
no nos toca decirnos te quiero
ni cuidar lo poco de dinero
que ha quedado en el cajón
en esta no
aunque duela tanto aceptarlo
y me quede con ganas de dar
lo que me quema el corazón
en esta no
nuestra historia nunca comenzó
tal vez en otra vida
pueda dart

In [69]:
# Save the final dataset
songs.to_csv('data/songs.csv', index=False)